In [2]:
#import required libraries
import geopandas as gpd
import xarray as xr
import pandas as pd
import os, glob
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.cm as cm

import rioxarray as rio
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.windows import Window, transform as rio_window_transform
from rasterio.windows import from_bounds
from rasterio.transform import Affine
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.features import rasterize
from shapely.geometry import box

import dask
import dask.array as da
from dask.delayed import delayed

from osgeo import gdal, ogr, osr

import seaborn as sns
import zipfile

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\4\ipykernel_173084\3466570486.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https

In [1]:
def rescale_and_clip_to_target(input_raster_path, target_raster_path, output_raster_path, compression, dtype):
    """
    Rescale a raster to match the resolution and clip it to the extent of another raster.

    Parameters:
    - input_raster_path: str, path to the input raster file (e.g., 'path/to/input_raster.tif')
    - target_raster_path: str, path to the target raster file (e.g., 'path/to/target_raster.tif')
    - output_raster_path: str, path to the output raster file (e.g., 'path/to/output_raster.tif')
    - compression: str, compression type (e.g., 'zstd', 'deflate', 'lwz')
    - dtype: str, output data type (e.g., 'int16')
    """
    
    with rasterio.open(target_raster_path) as target_src:
        # Get the extent and resolution of the target raster
        target_extent = target_src.bounds
        target_resolution = target_src.res[0]  # Assuming square pixels
    
    with rasterio.open(input_raster_path) as src:
        # Ensure the data type is the same as the target
        if src.dtypes[0] != dtype:
            raise ValueError(f"Input raster is not of type {dtype}")
        
        # Calculate the new transform and dimensions for the target extent
        new_transform = Affine(target_resolution, 0, target_extent.left, 0, -target_resolution, target_extent.top)
        new_width = int((target_extent.right - target_extent.left) / target_resolution)
        new_height = int((target_extent.top - target_extent.bottom) / target_resolution)
        
        # Read the data, resampling and clipping as necessary
        window = from_bounds(*target_extent, transform=src.transform)
        data = src.read(
            out_shape=(src.count, new_height, new_width),
            window=window,
            resampling=Resampling.nearest
        )
        
        # Update metadata
        profile = src.profile
        profile.update({
            'transform': new_transform,
            'width': new_width,
            'height': new_height,
            'res': (target_resolution, target_resolution),
            'dtype': dtype,
            'compress': f'{compression}'
        })
        
        # Write the resampled and clipped data to a new file
        with rasterio.open(output_raster_path, 'w', **profile) as dst:
            dst.write(data.astype(dtype))